In [1]:

from dotenv import load_dotenv
load_dotenv()

True

In [2]:
from langchain.chat_models import init_chat_model

llm = init_chat_model("openai:gpt-4o", temperature=0)
response = llm.invoke("Hola, como estas?")
response.text

'¡Hola! Estoy bien, gracias. ¿Y tú? ¿En qué puedo ayudarte hoy?'

In [3]:
response = llm.invoke("que clima hace en la sabana de bogota")
response.text

'La sabana de Bogotá, ubicada en el altiplano cundiboyacense de Colombia, tiene un clima de alta montaña, clasificado generalmente como un clima de páramo o subtropical de alta montaña. Las características principales del clima en esta región son:\n\n1. **Temperaturas**: Las temperaturas son relativamente frescas durante todo el año, con un promedio que oscila entre los 12°C y 15°C. Las noches pueden ser frías, con temperaturas que a veces descienden por debajo de los 10°C.\n\n2. **Precipitaciones**: La sabana de Bogotá experimenta dos temporadas de lluvias al año: una en abril-mayo y otra en octubre-noviembre. Los meses más secos suelen ser diciembre, enero, julio y agosto.\n\n3. **Humedad**: La humedad es generalmente alta debido a la altitud y la presencia de cuerpos de agua en la región.\n\n4. **Altitud**: La altitud promedio de la sabana es de aproximadamente 2,600 metros sobre el nivel del mar, lo que contribuye a las temperaturas frescas y a la baja presión atmosférica.\n\nEste 

In [11]:
system_prompt = """
Eres un asistente de ventas que ayuda a los clientes a encontrar los productos que necesitan.

Y tus productos son:
- Computadora
- Mouse
- Teclado
- Audifonos
- Mousepad
"""
messages = [
    ("system", system_prompt),
    ("user", "Dime los productos que ofreces en la tienda")
]
response = llm.invoke(messages)
response.text

'Claro, en nuestra tienda ofrecemos los siguientes productos:\n\n- Computadora\n- Mouse\n- Teclado\n- Audífonos\n- Mousepad\n\nSi necesitas más información sobre alguno de estos productos, no dudes en preguntar.'

In [17]:
from langchain_core.tools import tool
import requests

@tool("get_products", description="Get the products that the store offers filter by price")
def get_products():
    # Connnect with API
    """Get the products that the store offers"""
    response = requests.get("https://api.escuelajs.co/api/v1/products")
    products = response.json()
    return "".join([f"{product['title']} - {product['price']}" for product in products])


In [18]:
get_products.invoke({})

'Majestic Mountain Graphic T-Shirt - 44Classic Heather Gray Hoodie - 69Classic Black Hooded Sweatshirt - 79Classic Red Pullover Hoodie - 10Classic Comfort Fit Joggers - 25Classic Grey Hooded Sweatshirt - 90Classic Navy Blue Baseball Cap - 61Classic Red Jogger Sweatpants - 98Classic Comfort Drawstring Joggers - 79Classic Blue Baseball Cap - 86Classic Black Baseball Cap - 58Classic Red Baseball Cap - 35Classic Olive Chino Shorts - 84Classic High-Waisted Athletic Shorts - 43Classic White Crew Neck T-Shirt - 39Classic White Tee - Timeless Style and Comfort - 73Classic Black T-Shirt - 35Sleek White & Orange Wireless Gaming Controller - 69Sleek Wireless Headphone & Inked Earbud Set - 44Sleek Comfort-Fit Over-Ear Headphones - 28Efficient 2-Slice Toaster - 48Sleek Modern Laptop with Ambient Lighting - 43Sleek Wireless Computer Mouse - 10Sleek Modern Laptop for Professionals - 97Stylish Red & Silver Over-Ear Headphones - 39Sleek Smartwatch with Vibrant Display - 16Sleek Mirror Finish Phone Case

In [22]:
@tool("get_weather", description="Get the weather of a city")
def get_weather(city: str):
    response = requests.get(f"https://geocoding-api.open-meteo.com/v1/search?name={city}&count=1")
    data = response.json()
    latitude = data["results"][0]["latitude"]
    longitude = data["results"][0]["longitude"]
    response = requests.get(f"https://api.open-meteo.com/v1/forecast?latitude={latitude}&longitude={longitude}&current_weather=true")
    data = response.json()
    response = f"The weather in {city} is {data["current_weather"]["temperature"]}C with {data["current_weather"]["windspeed"]}km/h of wind."
    return response

get_weather.invoke({"city": "bogota"})

'The weather in bogota is 15.3C with 4.0km/h of wind.'

In [28]:
system_prompt = """
Eres un asistente de ventas que ayuda a los clientes a encontrar los productos que necesitan y dar el clima de la ciudad

Tus tools son:
- get_products: para obtener los productos que ofreces en la tienda
- get_weather: para obtener el clima de la ciudad
"""
messages = [
    ("system", system_prompt),
    ("user", "dime los productos que ofreces en la tienda")
]
llm_with_tools = llm.bind_tools([get_products, get_weather])
response = llm_with_tools.invoke(messages)
response.tool_calls
print(response.tool_calls)

[{'name': 'get_products', 'args': {}, 'id': 'call_q9zFRfRW2W0pqo32Pc8CU4Ta', 'type': 'tool_call'}]
